### Convert to "utf-8" encoded

In [3]:
! in2csv -e iso-8859-1 'RePORTER_PRJ_C_FY2016.csv' > sandbox.csv

### Get a numbered list of the columns

In [4]:
! csvcut -n sandbox.csv

  1: APPLICATION_ID
  2: ACTIVITY
  3: ADMINISTERING_IC
  4: APPLICATION_TYPE
  5: ARRA_FUNDED
  6: AWARD_NOTICE_DATE
  7: BUDGET_START
  8: BUDGET_END
  9: CFDA_CODE
 10: CORE_PROJECT_NUM
 11: ED_INST_TYPE
 12: FOA_NUMBER
 13: FULL_PROJECT_NUM
 14: FUNDING_ICs
 15: FUNDING_MECHANISM
 16: FY
 17: IC_NAME
 18: NIH_SPENDING_CATS
 19: ORG_CITY
 20: ORG_COUNTRY
 21: ORG_DEPT
 22: ORG_DISTRICT
 23: ORG_DUNS
 24: ORG_FIPS
 25: ORG_NAME
 26: ORG_STATE
 27: ORG_ZIPCODE
 28: PHR
 29: PI_IDS
 30: PI_NAMEs
 31: PROGRAM_OFFICER_NAME
 32: PROJECT_START
 33: PROJECT_END
 34: PROJECT_TERMS
 35: PROJECT_TITLE
 36: SERIAL_NUMBER
 37: STUDY_SECTION
 38: STUDY_SECTION_NAME
 39: SUBPROJECT_ID
 40: SUFFIX
 41: SUPPORT_YEAR
 42: DIRECT_COST_AMT
 43: INDIRECT_COST_AMT
 44: TOTAL_COST
 45: TOTAL_COST_SUB_PROJECT


### Extract just the grant type (activity), full project number, investigator IDs, name and total cost

In [5]:
! csvcut -c 2,13,29,30,44 sandbox.csv > temp && mv temp sandbox-cut.csv

### Take a look at the grant types

In [6]:
! csvcut -c 1 sandbox-cut.csv | csvlook | head

|------------|
|  ACTIVITY  |
|------------|
|  K23       |
|  R01       |
|  D43       |
|  R01       |
|  R01       |
|  P41       |
|  U01       |


### Remove any grant type that is not RO1

In [7]:
! csvgrep -c 1 -r 'R01|DP2|R23|R29|R37' sandbox-cut.csv > sandbox-grant-types.csv

### Take a look at the grant type

In [8]:
! csvcut -c 1 sandbox-grant-types.csv | csvlook | head

|------------|
|  ACTIVITY  |
|------------|
|  R01       |
|  R01       |
|  R01       |
|  R01       |
|  R01       |
|  R01       |
|  DP2       |


### Load pandas

In [1]:
import pandas

### Load the data into a dataframe

In [2]:
nih_df = pandas.read_csv("./sandbox-grant-types.csv")
nih_df.head()

,ACTIVITY,FULL_PROJECT_NUM,PI_IDS,PI_NAMEs,TOTAL_COST
0,R01,4R01NS085165-04,7017365;,"POLSTER, BRIAN M;",335781
1,R01,1R01MH111502-01,2275890 (contact); 8742217; 6139020;,"DULAC, CATHERINE G (contact); REGEV, AVIV ; ZH...",1593922
2,R01,5R01MH105329-02,9868481; 2063694 (contact);,"CHEN, XUESONG ; GEIGER, JONATHAN DAVID (contact);",294755
3,R01,5R01DK105393-02,9335858;,"QI, LING ;",202611
4,R01,4R01AG042165-05,10940848; 1897156 (contact);,"LOZANO, ANDRES M.; LYKETSOS, CONSTANTINE G (co...",472366


### Drop any rows that don't have a total cost

In [3]:
nih_df2 = nih_df.dropna()

### Collect all of the IDs into one long string and remove the word '(contact)'

In [5]:
ids = ""
for index, row in nih_df2.iterrows():
    ids += row['PI_IDS']
    
noContact = ids.replace("(contact)","")

### Split the string into a list on each semicolon

In [6]:
listOfIds = noContact.split(";")
listOfIds[:10]

['7017365',
 '2275890 ',
 ' 8742217',
 ' 6139020',
 '9868481',
 ' 2063694 ',
 '9335858',
 '10940848',
 ' 1897156 ',
 '1874169']

### Remove traling whitespace from each name

In [7]:
listOfIds = [x.strip(" ") for x in listOfIds]
len(listOfIds)

31245

### Remove duplicates

In [8]:
uniqueIds = [];
for item in listOfIds:
    if item not in uniqueIds:
        if len(item) > 1:
            uniqueIds.append(item)
            
len(uniqueIds)

21923

### Create a new data frame to store our records

In [16]:
total_df = pandas.DataFrame({
        'id': uniqueIds
    })

total_df['total-amount'] = 0
total_df['number-of-grants'] = 0
total_df['grant-types'] = ""
total_df['grant-numbers'] = ""

total_df.head(10)

,id,total-amount,number-of-grants,grant-types,grant-numbers
0,7017365,0,0,,
1,2275890,0,0,,
2,8742217,0,0,,
3,6139020,0,0,,
4,9868481,0,0,,
5,2063694,0,0,,
6,9335858,0,0,,
7,10940848,0,0,,
8,1897156,0,0,,
9,1874169,0,0,,


### Total up the cost for each instance of the names

In [17]:
for i, totalElem in total_df.iterrows():
    thisId = totalElem['id']
    if thisId != 'nan':
        id_correct_indexes = nih_df2['PI_IDS'].str.contains(thisId, case=True)
        id_correct_rows = nih_df2[id_correct_indexes]
        summed = id_correct_rows['TOTAL_COST'].sum()
        number_of_grants = len(id_correct_rows['PI_IDS'])
        types = id_correct_rows['ACTIVITY'].str.cat(sep=' + ')
        grant_numbers = id_correct_rows['FULL_PROJECT_NUM'].str.cat(sep=' + ')
        total_df.set_value(i, 'total-amount', summed)
        total_df.set_value(i, 'number-of-grants', number_of_grants)
        total_df.set_value(i, 'grant-types', types)
        total_df.set_value(i, 'grant-numbers', grant_numbers)

total_df.head(10)

,id,total-amount,number-of-grants,grant-types,grant-numbers
0,7017365,335781,1,R01,4R01NS085165-04
1,2275890,3506620,4,R01 + R01 + R01 + R01,1R01MH111502-01 + 4R01DC013087-04 + 1R01MH1130...
2,8742217,2806468,2,R01 + R01,1R01MH111502-01 + 1R01HG009276-01
3,6139020,3114521,3,R01 + R01 + R01,1R01MH111502-01 + 4R01GM105637-04 + 1R01MH1130...
4,9868481,607505,2,R01 + R01,5R01MH105329-02 + 5R01MH100972-03
5,2063694,607505,2,R01 + R01,5R01MH105329-02 + 5R01MH100972-03
6,9335858,1260150,5,R01 + R01 + R01 + R01 + R01,5R01DK105393-02 + 5R01GM113188-02 + 1R01DK1111...
7,10940848,472366,1,R01,4R01AG042165-05
8,1897156,4397431,3,R01 + R01 + R01,4R01AG042165-05 + 1R01AG052510-01 + 1R01AG0518...
9,1874169,2191656,3,R01 + R01 + R01,5R01MH101118-03 + 5R01MH104647-03 + 1R01MH1121...


### Count rows in new dataframe

In [19]:
len(total_df)

21923

### Create a list of dicts containing names and ids

In [22]:
names_and_ids = [];

for index, row in nih_df2.iterrows():
    row_ids = row['PI_IDS'].split(';')
    row_names = row['PI_NAMEs'].split(';')
    if len(row_ids) > 2:
        contact_id = filter(lambda x: '(contact)' in x, row_ids)[0].strip("(contact)").strip()
        contact_name = filter(lambda x: '(contact)' in x, row_names)[0].strip("(contact)").strip()
    else:
        contact_id = row_ids[0].strip()
        contact_name = row_names[0].strip()
    this_dict = {'name': contact_name, 'id': contact_id }
    names_and_ids.append(this_dict)
        
print names_and_ids[:10]

[{'name': 'POLSTER, BRIAN M', 'id': '7017365'}, {'name': 'DULAC, CATHERINE G', 'id': '2275890'}, {'name': 'GEIGER, JONATHAN DAVID', 'id': '2063694'}, {'name': 'QI, LING', 'id': '9335858'}, {'name': 'LYKETSOS, CONSTANTINE G', 'id': '1897156'}, {'name': 'CLARKE, GREGORY N', 'id': '1874169'}, {'name': 'ANDREWS, JASON RANDOLPH', 'id': '10518018'}, {'name': 'FISHER, STEVEN K', 'id': '1879326'}, {'name': 'MANFREDI, GIOVANNI', 'id': '3150937'}, {'name': 'CICHEWICZ, ROBERT HENRY', 'id': '8769887'}]


### Create a new dataframe containing name and id columns, with only unique values

In [23]:
name_id_df = pandas.DataFrame(names_and_ids)
unique_name_id_df = name_id_df.drop_duplicates()
unique_name_id_df.head(10)

,id,name
0,7017365,"POLSTER, BRIAN M"
1,2275890,"DULAC, CATHERINE G"
2,2063694,"GEIGER, JONATHAN DAVID"
3,9335858,"QI, LING"
4,1897156,"LYKETSOS, CONSTANTINE G"
5,1874169,"CLARKE, GREGORY N"
6,10518018,"ANDREWS, JASON RANDOLPH"
7,1879326,"FISHER, STEVEN K"
8,3150937,"MANFREDI, GIOVANNI"
9,8769887,"CICHEWICZ, ROBERT HENRY"


### Count rows in new dataframe

In [24]:
len(unique_name_id_df)

18818

### Merge the two dataframes

In [25]:
merge_df = pandas.merge(total_df, unique_name_id_df, on="id", how="outer")
merge_df.head(10)

,id,total-amount,number-of-grants,grant-types,grant-numbers,name
0,7017365,335781,1,R01,4R01NS085165-04,"POLSTER, BRIAN M"
1,2275890,3506620,4,R01 + R01 + R01 + R01,1R01MH111502-01 + 4R01DC013087-04 + 1R01MH1130...,"DULAC, CATHERINE G"
2,8742217,2806468,2,R01 + R01,1R01MH111502-01 + 1R01HG009276-01,"REGEV, AVIV"
3,6139020,3114521,3,R01 + R01 + R01,1R01MH111502-01 + 4R01GM105637-04 + 1R01MH1130...,"ZHUANG, XIAOWEI"
4,9868481,607505,2,R01 + R01,5R01MH105329-02 + 5R01MH100972-03,NaN
5,2063694,607505,2,R01 + R01,5R01MH105329-02 + 5R01MH100972-03,"GEIGER, JONATHAN DAVID"
6,9335858,1260150,5,R01 + R01 + R01 + R01 + R01,5R01DK105393-02 + 5R01GM113188-02 + 1R01DK1111...,"QI, LING"
7,10940848,472366,1,R01,4R01AG042165-05,NaN
8,1897156,4397431,3,R01 + R01 + R01,4R01AG042165-05 + 1R01AG052510-01 + 1R01AG0518...,"LYKETSOS, CONSTANTINE G"
9,1874169,2191656,3,R01 + R01 + R01,5R01MH101118-03 + 5R01MH104647-03 + 1R01MH1121...,"CLARKE, GREGORY N"


### Sort by total-amount

In [26]:
sorted_df = merge_df.sort('total-amount', ascending=False)
sorted_df.head(10)

,id,total-amount,number-of-grants,grant-types,grant-numbers,name
594,7370713,8049330,3,R01 + R01 + R01,4R01AG043962-03 + 1R01AG053952-01 + 1R01AG0497...,"BURNS, JEFFREY MURRAY"
11895,1883846,6991146,2,R01 + R01,4R01DA016575-14 + 4R01DA001411-42,"JOHNSTON, LLOYD D"
6432,2083909,6561002,5,R01 + R01 + R01 + R01 + R01,3R01HD052646-08S1 + 4R01HD052646-08 + 2R01HD06...,"SASTRY, NARAYAN"
1525,7861723,6517026,3,R01 + R01 + R01,5R01AG046179-02 + 4R01NS065667-09 + 1R01NS0957...,"BATEMAN, RANDALL J"
3100,1895462,6290349,4,R01 + R37 + R01 + R01,4R01MH075957-09 + 4R37DA035377-04 + 5R01MH0863...,"DEISSEROTH, KARL A."
14867,8658504,6274848,4,R01 + R01 + R01 + R01,1R01AG054029-01 + 5R01AG046396-03 + 1R01DC0142...,"JOHNSON, KEITH A."
11167,2059843,5949179,3,R01 + R01 + R01,5R01AG047992-03 + 3R01AG047992-03S1 + 1R01AG05...,NaN
11765,1862204,5695022,3,R01 + R01 + R01,5R01DE016148-11 + 2R01DE014899-13 + 3R01DE0148...,"MARAZITA, MARY L."
9299,8666855,5426324,5,R01 + R01 + R01 + R01 + R01,1R01AG054076-01 + 2R01NS017950-33 + 5R01AG0496...,"SESHADRI, SUDHA"
5756,6902290,5404777,3,R01 + R01 + R01,1R01AG053952-01 + 4R01DK095172-05 + 1R01CA1967...,"ERICKSON, KIRK I"


### Export to a new csv

In [27]:
sorted_df.to_csv('sandbox-output-ids-names.csv', sep=',')

### Count the number of lines in this new file

In [29]:
! wc -l sandbox-output-ids-names.csv

   21924 sandbox-output-ids-names.csv


### Get the column numbers

In [30]:
! csvcut -n sandbox-output-ids-names.csv

  1: 
  2: id
  3: total-amount
  4: number-of-grants
  5: grant-types
  6: grant-numbers
  7: name


### Drop the first column and the names

In [31]:
! csvcut -c 2,3,4,5,6,7 sandbox-output-ids-names.csv > temp && mv temp sandbox-output-ids-names.csv